In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import datetime
from datetime import timedelta
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
import os

In [3]:
# os.chdir('/media/minwookim/9C33-6BBD/WorkSpace/Next_Ez_jejudo/data/골프장데이터')리눅스

In [ ]:
df = pd.read_csv('골프장예약만데이터ver0303.csv')
df.head(3)

### 1. 부킹시간대
01 ~ 07 = [1,2,3,4,5,6,7]

08 ~ 12 = [8,9,10,11,12]

13 ~ 18 = [13,14,15,16,17,18]

19 ~ 24 = [19,20,21,22,23,24]

### 2. 사전예약(취소) 통계
당일예약

1~2일전

3~7일전

8~15일

16일~


### 3.사용일 통계

시간대/일자

06시~08시

09시 ~ 10시

11시~12시

13시 ~


### 4. 가격대 별 예약(취소) 통계
- 도민, 비도민 나누기
90,000 이하

90,000~,110000

110,000~120,000

120,000~130,000

130,000~140,000

140,000~160,000

160,000 ~ 180,000
180,000 ~ 200,000
200,000원 이상 


### 5. 전화 연결 통계

시간대/일자

~ 09시 

09시 ~ 12시

13시 ~ 18시

19시 ~ 24시


In [97]:
df_cancel = pd.read_csv('/media/minwookim/9C33-6BBD/WorkSpace/Next_Ez_jejudo/data/골프장데이터/제주골프장_예약취소_2112-2202.csv',encoding='euc-KR',header=None)

In [98]:
col_list = '''골프장명
골프장지역명
골프장멤버십값
골프장유형값
사용일자
사용시간값
사용유형명
거주자금액
비거주자금액
결제수단명
총인원수
거주자인원수
비거주자인원수
예약일자
예약시간
취소일자
취소시간
판매업체명
예약자성별코드
예약자연령대코드
예약자거주지값
예약자평균값
예약자골프경력값
예약자평균사용값
예약자사용성향값'''.split('\n')

In [18]:
len(col_list)

25

In [19]:
len(df_cancel.columns)

25

In [99]:
df_cancel.columns=col_list

In [6]:
df_cancel.총인원수.unique()

array([4])

In [100]:
df_cancel.columns

Index(['골프장명', '골프장지역명', '골프장멤버십값', '골프장유형값', '사용일자', '사용시간값', '사용유형명',
       '거주자금액', '비거주자금액', '결제수단명', '총인원수', '거주자인원수', '비거주자인원수', '예약일자', '예약시간',
       '취소일자', '취소시간', '판매업체명', '예약자성별코드', '예약자연령대코드', '예약자거주지값', '예약자평균값',
       '예약자골프경력값', '예약자평균사용값', '예약자사용성향값'],
      dtype='object')

In [19]:
df_cancel.취소일자[0]

20211201

In [25]:
str(df_cancel.취소일자[0])[:4]

'2021'

In [101]:
def timemiuns(x):
    return (datetime.date(int(str(x)[:4]) ,int(str(x)[4:-2].lstrip('0')) ,int(str(x)[-2:].lstrip('0'))))


In [44]:
(datetime.date(int(str(df_cancel.취소일자[0])[:4]) ,int(str(df_cancel.취소일자[0])[4:-2].lstrip('0')) ,int(str(df_cancel.취소일자[0])[-2:].lstrip('0')))
 -
 datetime.date(int(str(df_cancel.예약일자[0])[:4]) ,int(str(df_cancel.예약일자[0])[4:-2].lstrip('0')) ,int(str(df_cancel.예약일자[0])[-2:].lstrip('0')))
 ).days

14

In [102]:
df_cancel.사전예약취소=(df_cancel.취소일자.apply(lambda x : timemiuns(x)) - df_cancel.예약일자.apply(lambda x: timemiuns(x)))
df_cancel.사전예약취소 = df_cancel.사전예약취소.apply(lambda x : x.days)

/tmp/ipykernel_12639/4028847853.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_cancel.사전예약취소=(df_cancel.취소일자.apply(lambda x : timemiuns(x)) - df_cancel.예약일자.apply(lambda x: timemiuns(x)))


In [103]:
df_cancel.사전예약취소.value_counts()

0     578
1     326
2     167
3     142
4     123
5      74
6      69
7      50
8      41
9      21
12     18
11     17
10     16
13     14
14     11
15      9
20      8
17      8
16      7
19      7
18      6
22      3
21      1
23      1
dtype: int64

<function __main__.<lambda>(x)>

In [33]:
datetime.date(int(str(df_cancel.예약일자[0])[:4]) ,int(str(df_cancel.예약일자[0])[4:-2].lstrip('0')) ,int(str(df_cancel.예약일자[0])[-2:].lstrip('0')))

datetime.date(2021, 11, 17)

## 취소 

In [8]:
df_cancel.취소일자[0]-df_cancel.예약일자[0]

84

In [34]:
df_cancel.취소일자[0]

20211201

In [35]:
df_cancel.예약일자[0]

20211117

In [79]:
df_cancel.사전예약취소

0       14
1        6
2        0
3        1
4        2
        ..
1712     8
1713     2
1714     2
1715     2
1716     8
Length: 1717, dtype: int64

##
당일예약

1~2일전

3~7일전

8~15일

16일~

In [104]:
df_cancel.사전예약취소 = df_cancel.사전예약취소.apply(lambda x : '당일예약' if (x in [0]) else
                                          '1~2일전' if (x in [1,2]) else
                                          '3~7일전' if (x in [3,4,5,6,7]) else
                                          '8~15일전' if (x in [8,9,10,11,12,13,14,15]) else '16일~' )

In [106]:
df_cancel.사전예약취소.value_counts()

당일예약      578
1~2일전     493
3~7일전     458
8~15일전    147
16일~       41
dtype: int64

In [82]:
df_cancel.사전예약취소.value_counts()

1~2일전    1139
당일예약      578
dtype: int64

In [ ]:
df_cancel['월'] = df_cancel['사용일자'].apply(lambda x: str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [3,4,5]) else 
                                    str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [6,7,8]) else 
                                   str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [9,10,11]) else 
                                    str(x)[4:-2].lstrip('0')+'월'  if ( int(str(x)[4:-2].lstrip('0')) in [12,1,2]) else '미분류')

## 당일예약

## 참고 

In [ ]:
df['월'] = df['사용일자'].apply(lambda x: str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [3,4,5]) else 
                                    str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [6,7,8]) else 
                                   str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [9,10,11]) else 
                                    str(x)[4:-2].lstrip('0')+'월'  if ( int(str(x)[4:-2].lstrip('0')) in [12,1,2]) else '미분류')

In [26]:
days=datetime.date(2021,12,1)-datetime.date(2021,10,1)

년 int(str(x)[:4]) 

월 int(str(x)[4:-2].lstrip('0')) 

일 int(str(x)[-2:].lstrip('0')) 